In [1]:
from google.cloud import storage, aiplatform

In [2]:
ARTIFACT_FILENAME = "model.pkl"
BUCKET = "iris-classification"
ARTIFACT_URI = f"gs://{BUCKET}"
MODEL = "iris"
ENDPOINT = "iris"

# the serving container image
# https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
DEPLOY_IMAGE = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

PROJECT = "cloud4us-gcp-3fbr5mcqluov7itt4"
LOCATION = "europe-west1"

# create Cloud Storage bucket

In [3]:
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET)
new_bucket = storage_client.create_bucket(bucket, location="europe-west1")

Conflict: 409 POST https://storage.googleapis.com/storage/v1/b?project=cloud4us-gcp-3fbr5mcqluov7itt4&prettyPrint=false: Your previous request to create the named bucket succeeded and you already own it.

# copy local .pkl file to Cloud Storage

In [4]:
!gsutil cp {ARTIFACT_FILENAME} {ARTIFACT_URI}

Copying file://model.pkl [Content-Type=application/octet-stream]...
/ [1 files][172.2 KiB/172.2 KiB]                                                
Operation completed over 1 objects/172.2 KiB.                                    


# init aiplatform

In [5]:
aiplatform.init(project=PROJECT, location=LOCATION)

# upload model to Vertex AI

In [8]:
%%time

model = aiplatform.Model.upload(
    display_name=MODEL,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE
)

Creating Model
Create Model backing LRO: projects/704710201124/locations/europe-west1/models/4723202885762940928/operations/5935745408385941504
Model created. Resource name: projects/704710201124/locations/europe-west1/models/4723202885762940928@1
To use this Model in another session:
model = aiplatform.Model('projects/704710201124/locations/europe-west1/models/4723202885762940928@1')
CPU times: user 53.9 ms, sys: 4.88 ms, total: 58.8 ms
Wall time: 5.18 s


# deploy model to endpoint

In [13]:
%%time

endpoint=model.deploy(deployed_model_display_name=ENDPOINT)

Creating Endpoint
Create Endpoint backing LRO: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408/operations/6305037279295438848
Endpoint created. Resource name: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/704710201124/locations/europe-west1/endpoints/4697342372277649408')
Deploying model to Endpoint : projects/704710201124/locations/europe-west1/endpoints/4697342372277649408
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408/operations/7512001979430731776
Endpoint model deployed. Resource name: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408
CPU times: user 275 ms, sys: 52.2 ms, total: 327 ms
Wall time: 13min 56s


# 🧹 clean up

In [15]:
endpoint.delete(force=True)  # force=True to undeploy all models before deleting endpoint

Undeploying Endpoint model: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408
Undeploy Endpoint model backing LRO: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408/operations/6327555277432291328
Endpoint model undeployed. Resource name: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408
Deleting Endpoint : projects/704710201124/locations/europe-west1/endpoints/4697342372277649408
Delete Endpoint  backing LRO: projects/704710201124/locations/europe-west1/operations/441353862993936384
Endpoint deleted. . Resource name: projects/704710201124/locations/europe-west1/endpoints/4697342372277649408


In [16]:
model.delete()

Deleting Model : projects/704710201124/locations/europe-west1/models/4723202885762940928
Delete Model  backing LRO: projects/704710201124/locations/europe-west1/operations/1805944550087196672
Model deleted. . Resource name: projects/704710201124/locations/europe-west1/models/4723202885762940928


In [17]:
# Delete Cloud Storage bucket
!gcloud storage rm --recursive {ARTIFACT_URI}

Removing objects:
Removing gs://iris-classification/model.pkl#1665937786782425...                
  Completed 1/1                                                                
Removing Buckets:
Removing gs://iris-classification/...                                          
  Completed 1/1                                                                
